## Missing Values

Missing data, or missing values, occur when __no data__ / __no value__ is stored for certain observations within a variable. 

Incomplete data is an unavoidable problem in most data sources, and may have a significant impact on the conclusions that can be derived from the data. 

### Why is data missing?

The source of missing data can be very different. These are just a few examples:

- A value is missing because it was forgotten, lost or not stored properly
- For a certain observation, the value does not exist
- The value can't be known or identified

In many organisations, information is collected into a form by a person talking with a client on the phone, or alternatively, by customers filling forms online. Often, the person entering the data does not complete all the fields in the form. Many of the fields are not compulsory, which may lead to missing values.

The reasons for omitting the information can vary: perhaps the person does not want to disclose some information, for example income, or they do not know the answer, or the answer is not applicable for a certain circumstance, or on the contrary, the person in the organisation wants to spare the customer some time, and therefore omits asking questions they think are not so relevant.

There are other cases where the value for a certain variable does not exist. For example, in the variable 'total debt as percentage of total income' (very common in financial data), if the person has no income, then the total percentage of 0 does not exist, and therefore it will be a missing value.

It is important to understand **how the missing data are introduced in the dataset**, that is, the **mechanisms** by which missing information is introduced in a dataset. Depending on the mechanism, we may choose to process the missing values differently. In addition, by knowing the source of missing data, we may choose to take action to control that source and decrease the amount of missing information looking forward during data collection.


### Missing Data Mechanisms

There are 3 mechanisms that lead to missing data, 2 of them involve missing data randomly or almost-randomly, and the third one involves a systematic loss of data.

#### Missing Completely at Random, MCAR:

A variable is missing completely at random (MCAR) if the probability of being missing is the same for all the observations. 
When data is MCAR, there is absolutely no relationship between the data missing and any other values, observed or missing, within the dataset. In other words, those missing data points are a random subset of the data. There is nothing systematic going on that makes some data more likely to be missing than other. If values for observations are missing completely at random, then disregarding those cases would not bias the inferences made.


#### Missing at Random, MAR: 

MAR occurs when there is a relationship between the propensity of missing values and the observed data. In other words, the probability of an observation being missing depends on available information (i.e., other variables in the dataset). For example, mechanical sensors are more likely to fail than electronic ones. For that reason, data imputed from mechanical sensors can be missing more often. 

In a situation like the above, if we decide to proceed with the variable with missing values, we might benefit from including the type of sensor to control the bias.


#### Missing Not at Random, MNAR: 

Missing data is not at random (MNAR) when there is a mechanism or a reason why missing values are introduced in the dataset. For example, when a financial company asks for bank and identity documents from customers in order to prevent identity fraud, typically, fraudsters impersonating someone else will not upload documents, because they don't have them, because they are fraudsters. Therefore, there is a systematic relationship between the missing documents and the target we want to predict: fraud.

Understanding the mechanism by which data is missing is important to decide which methods to use to impute the missing values.

====================================================================================================

## In this class, we will:

- Learn how to detect and quantify missing values

- Try to identify the 3 different mechanisms of missing data introduction

We will use the toy Loan dataset and the Titanic dataset.

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# to display the total number columns present in the dataset
pd.set_option('display.max_columns', None)

In [0]:
# let's load the titanic dataset
data = pd.read_csv('/dbfs/FileStore/CDS2024/titanic.csv')
data.head()

In python, the missing values are stored as NaN, see for example the first row for the variable 'Cabin'.
We can quantify the total number of missing values using the **isnull** method plus the sum method on the dataframe

In [0]:
data.isnull().sum()

There are 177 missing values for Age, 687 for Cabin and 2 for Embarked.

Alternatively, we can use the mean method after isnull to visualize the percentage of missing values for each variable

In [0]:
data.isnull().mean()

There are missing data in the variables Age (20% missing), Cabin - in which the passenger was traveling - (77% missing), and Embarked - the port from which the passenger got into the Titanic - (~0.2%  missing).

## Mechanisms of Missing Data

### Missing data Not At Random (MNAR): Systematic missing values

In the Titanic dataset, both the missing values of the variables **age** and **cabin** were introduced systematically. For many of the people who did not survive, the **age** they was or the **cabin** they were traveling in, could not be established. The people who survived could be otherwise asked for that information.

Can we infer this by looking at the data?

In a situation like this, we could expect a greater number of missing values for people who did not survive.

Let's have a look.

Let's create a binary variable that indicates whether the value of cabin is missing

In [0]:
data['cabin_null'] = np.where(data['Cabin'].isnull(), 1, 0)

Let's evaluate the percentage of missing values in cabin for the people who survived vs the non-survivors.

The variable Survived takes the value 1 if the passenger survived, or 0 otherwise.

In [0]:
# Group data by Survived vs Non-Survived and find the percentage of nulls for cabin
data.groupby(['Survived'])['cabin_null'].mean()

In [0]:
# another way of doing the above

data['Cabin'].isnull().groupby(data['Survived']).mean()

We observe that the percentage of missing values is higher for people who did not survive (87%), respect to people who survived (60%). This finding is aligned with our hypothesis that the data is missing because after people died, the information could not be retrieved.

**Note**: Having said this, to truly underpin whether the data is missing not at random, we would need to get extremely familiar with the way data was collected. Analyzing datasets, can only point us in the right direction or help us build assumptions.

**Excercise:** Let's do the same for the variable Age. What can we conclude?

In [0]:
# First we create a binary variable to indicates whether the value of Age is missing

# Then look at the mean in the different survival groups:


**Conclusion:**

### Missing data Completely At Random (MCAR)

In the titanic dataset, there are also missing values for the variable Embarked. Let's have a look.

In [0]:
data[data['Embarked'].isnull()]

These 2 women were traveling together, Miss Icard was the maid of Mrs Stone.

A priori, it does not seem to be an indication that the missing information in the variable Embarked is depending on any other variable, and the fact that these women survived means that they could have been asked for this information.

Very likely the values were lost at the time of building the dataset.

If these values are MCAR, the probability of data being missing for these 2 women is the same as the probability for values to missing for any other person on the titanic. Of course this will be hard, if possible at all, to prove. But I hope this serves as a demonstration.

### Missing data at Random (MAR)

For this example, I will use the loan book toy dataset from this ficticious peer to peer lending company.

We will look at the variables employment and years in employment, both declared by the borrowers at the time of applying for a loan. 

In this example, data missing in employment are associated with data missing in time in employment.

In [0]:
# Let's use the Loan dataset and load the columns of interest for this exercise
data = pd.read_csv('/dbfs/FileStore/CDS2024/loan.csv', usecols=['employment', 'time_employed'])
data.head()

In [0]:
# let's check the percentage of missing data

data.isnull().mean()

We see that both variables have almost the same percentage of missing observations.

Let's inspect the different employment types

In [0]:
# number of different employments
n_employments = len(data['employment'].unique())
print('Number of employments: ' + str(n_employments))

# Examples of employments
data['employment'].unique()

We observe the missing information (nan), and several different employments of the people.

Let's inspect the variable time employed.

In [0]:
data['time_employed'].unique()

The customer can't enter a value for employment time if they are not employed. They could be students, retired, self-employed, or work in the house. But we can see how these 2 variables are related to each other.

Let's calculate the proportion of missing data for the time_employed variable for customers who declared employment

In [0]:
# customers who declared employment
t = data[~data['employment'].isnull()]

# percentage of missing data in time employed
print(t['time_employed'].isnull().mean(), '->' , round(t['time_employed'].isnull().mean()*100, 2), '%')

**Exercise:** Let´s do the same for those who did not report employment

In [0]:
# customers who did not declare employment

# percentage of missing data in time employed


**Conclusion:**

# Now let's see how we can deal with missing data.

## Complete Case Analysis


Complete-case analysis (CCA), also called "list-wise deletion" of cases, consists in **discarding** observations where values in **any** of the variables are missing. Complete Case Analysis means literally analysing only those observations for which there is information in **all** of the variables in the dataset. 

### Which variables can I impute with CCA?

CCA can be applied to both categorical and numerical variables.


### Assumptions

CCA works well when the data are missing completely at random (MCAR). In fact, we should use CCA only if we have reasons to believe that data is missing at random, and not otherwise. When data is MCAR, excluding observations with missing information is in essence the same as randomly excluding some observations from the dataset. Therefore the dataset after CCA is a fair representation of the original dataset. 


### Advantages

- Easy to implement
- No data manipulation required
- Preserves variable distribution (if data is MCAR, then the distribution of the variables of the reduced dataset should match the distribution in the original dataset)

### Disadvantages

- It can exclude a large fraction of the original dataset (if missing data is abundant)
- Excluded observations could be informative for the analysis (if data is not missing at random)
- CCA will create a biased dataset if the complete cases differ from the original data (e.g., when missing information is in fact MAR or NMAR and not missing at random).
- When using our models in production, the model will not know how to handle missing data

### When to use CCA

- Data is missing completely at random
- No more than 5% of the total dataset contains missing data

In practice, CCA may be an acceptable method when the amount of missing information is small. Unfortunately, there is no rule of thumb to determine how much missing data is small or negligible. However, as general guidance, if the total amount of missing data is ~5% of the original dataset or less, CCA is a viable option.

In many real life datasets, the amount of missing data is never small, and therefore CCA is typically never an option.

### CCA and models in production

When using CCA, we remove all observations that contain missing information. However, the data that we want to score with our model, may indeed contain missing information. This will pose a problem when using our model in live systems, or as we call it, when putting or models into production: when an observation contains missing data, the model will not be able to handle it. 

In order to avoid this problem, when putting models into production we need to do 1 of 2 things: either we do not score observations with missing data, or we replace the missing values by another number. We can choose any from the imputation techniques that we will discuss in the following lectures to replace NA in the data to be scored.

## In this section:

We will use the House Prices dataset to demonstrate how to perform Complete Case Analysis.

In [0]:
# let's load the House Prices dataset and explore its shape (rows and columns)
data = pd.read_csv('/dbfs/FileStore/CDS2024/houseprice.csv')
data.shape

In [0]:
data.head()

In [0]:
# to show all the columns of the dataframe in the notebook
pd.set_option('display.max_columns', None)
data.head()

In [0]:
# We can calculate the percentage of missing values ('NA' - Not available) for each variable and select those with more than 0%.

vars_with_na = [var for var in data.columns if data[var].isnull().mean() > 0]
vars_with_na

In [0]:
# let's find out whether they are numerical or categorical
data[vars_with_na].dtypes

There are both numerical and categorical variables with missing observations. We can see from the variable types that some are float and some are object.

In [0]:
# let's have a look at the values of the variables with missing data

data[vars_with_na].head(10)

## Let's find out the percentage of observations missing per variable

1 - Calculating the percentage of missing  using the isnull() and mean() methods from pandas

In [0]:
data_na = data[vars_with_na].isnull().mean()

2 - Transforming the array into a dataframe

In [0]:
data_na = pd.DataFrame(data_na.reset_index())

3 - Adding column names to the dataframe

In [0]:
data_na.columns = ['variable', 'na_percentage']

4 - Ordering the dataframe according to percentage of na per variable

In [0]:
data_na.sort_values(by='na_percentage', ascending=False, inplace=True)

data_na

The first 6 variables contain a lot of missing information. So we can't use CCA if we consider those variables, as most of the observations in the dataset will be discarded. We could otherwise use CCA if we omit using those variables with a lot of NA.

For this demo, I will ignore the first 6 variables with a lot of missing data, and proceed with CCA in the remaining of the dataset.

Selecting variables with no or less than 5% NA

In [0]:
vars_cca = [var for var in data.columns if data[var].isnull().mean() < 0.05]
vars_cca

In [0]:
data.shape[1], len(vars_cca)

Calculating the percentage of observations with complete cases: i.e., with values for all the variables

In [0]:
# The method dropna(), discards the observations that contain na in any of the rows / columns

len(data[vars_cca].dropna()) / len(data)

**Exercise:** Let's create the complete case dataset. In other words, remove observations with NA in any variable.

In [0]:
# New data set will consider the columns in vars_cca and then drop na 
data_cca = data[vars_cca].dropna()

# .shape to check the size of both data sets
data.shape, data_cca.shape

Let's plot the histograms for all numerical variables in the complete case dataset.

In [0]:
data_cca.hist(bins=50, density=True ,figsize=(16, 16))
plt.show()

Let's check the distribution and the density of a few variables before and after CCA.

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)

# original data
data['GrLivArea'].hist(bins=50, ax=ax, density=True, color='red') # density = True: normalize variable

# data after cca, the argument alpha makes the color transparent, so we can see the overlay of the 2 distributions
data_cca['GrLivArea'].hist(bins=50, ax=ax, color='blue', density=True, alpha=0.8)

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)

# original data
data['GrLivArea'].plot.density(color='red')

# data after cca
data_cca['GrLivArea'].plot.density(color='blue')

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)

# original data
data['BsmtFinSF1'].hist(bins=50, ax=ax, density=True, color='red')

# data after cca
data_cca['BsmtFinSF1'].hist(bins=50, ax=ax, color='blue', density=True, alpha=0.8)

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)

# original data
data['BsmtFinSF1'].plot.density(color='red')

# data after cca
data_cca['BsmtFinSF1'].plot.density(color='blue')

As we can see from the above plots, the distribution of the selected numerical variables in the original and complete case dataset is very similar, which is what we expect from CCA if data is missing at random and only for a small proportion of the observations.

In the next cells I will explore the distribution of categorical variables. To do so, I will evaluate the percentage of observations that show each of the unique categories.

In [0]:
#The following function captures the percentage of observations for each category in the original and complete case dataset
# and puts them together in a new dataframe


def categorical_distribution(df, df_cca, variable):
    tmp = pd.concat(
        [
            # percentage of observations per category, original data
            df[variable].value_counts() / len(df),

            # percentage of observations per category, cca data
            df_cca[variable].value_counts() / len(df_cca)
        ],
        axis=1)

    # add column names
    tmp.columns = ['original', 'cca']

    return tmp

In [0]:
# run the function in a categorical variable
categorical_distribution(data, data_cca, 'BsmtQual')

In [0]:
categorical_distribution(data, data_cca, 'MasVnrType')

In [0]:
categorical_distribution(data, data_cca, 'SaleCondition')

As we can see from the output of the cells above, the distribution of houses in each of the categories, is very similar in the original and complete case dataset, which again, is what is expected if the data is missing completely at random, and the percentage of missing data is small.

# Now let's see another method to deal with missing data.

## Mean / Median imputation

Imputation is the act of replacing missing data with statistical estimates of the missing values. The goal of any imputation technique is to produce a **complete dataset** that can be used to train machine learning models.

Mean / median imputation consists of replacing all occurrences of missing values (NA) within a variable by the mean (if the variable has a Gaussian distribution) or median (if the variable has a skewed distribution).

**Note the following**:

- If a variable is normally distributed, the mean, median and mode, are approximately the same. Therefore, replacing missing values by the mean and the median are equivalent. Replacing missing data by the mode is not common practice for  numerical variables.
- If the variable is skewed, the mean is biased by the values at the far end of the distribution. Therefore, the median is a better representation of the majority of the values in the variable.
- For discrete variables casted as 'int' (to save memory), the mean may not be an integer, therefore the whole variable will be re-casted as 'float'. In order to avoid this behaviour, we can replace NA with the median instead. The median will inevitably be an integer / discrete value as well.


### Which variables can I impute with Mean / Median Imputation?

The mean and median can only be calculated on numerical variables, therefore these methods are suitable for continuous and discrete numerical variables only.


### Assumptions

- Data is missing completely at random (MCAR)
- The missing observations, most likely look like the majority of the observations in the variable (aka, the mean / median)

If data is missing completely at random, then it is fair to assume that the missing values, are most likely very close to the value of the mean or the median of the distribution, as these represent the most frequent / average observation.


### Advantages

- Easy to implement
- Fast way of obtaining complete datasets
- Can be integrated in production (during model deployment)

### Limitations

- Distortion of the original variable distribution
- Distortion of the original variance
- Distortion of the covariance with the remaining variables of the dataset

When replacing NA with the mean or median, the variance of the variable will be distorted if the number of NA is big respect to the total number of observations, leading to underestimation of the variance.

In addition, estimates of covariance and correlations with other variables in the dataset may also be affected. Mean / median imputation may alter intrinsic correlations since the mean / median value that now replaces the missing data will not necessarily preserve the relation with the remaining variables.

Finally, concentrating all missing values at the mean / median value, may lead to observations that are common occurrences in the distribution, to be picked up as outliers.


### When to use mean / median imputation?

- Data is missing completely at random
- No more than 5% of the variable contains missing data

Although in theory, the above conditions should be met to minimise the impact of this imputation technique, in practice, mean / median imputation is very commonly used, even in those cases when data is not MCAR and there are a lot of missing values. The reason behind this, is the simplicity of the technique.


### Final note

Replacement of NA with mean / median is widely used in the data science community and in various data science competitions. See for example the winning solution of the KDD 2009 cup: ["Winning the KDD Cup Orange Challenge with Ensemble Selection"]( http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf).

Typically, mean / median imputation is done together with adding a binary "missing indicator" variable to capture those observations where the data was missing (see lecture "Missing Indicator"), thus covering 2 angles: if the data was missing completely at random, this would be captured by the mean /median imputation, and if it wasn't this would be captured by the additional "missing indicator" variable. Both methods are extremely straight forward to implement, and therefore are a top choice in data science competitions.

In [0]:
# to split the datasets
from sklearn.model_selection import train_test_split

## Let's put into practice with Titanic dataset

In [0]:
# load the Titanic Dataset with a few variables for demonstration

data = pd.read_csv('/dbfs/FileStore/CDS2024/titanic.csv', usecols=['Age', 'Fare', 'Survived'])
data.head()

In [0]:
# let's look at the percentage of NA
data.isnull().mean()

The only variable with missing data is Age, with ~20% of missing observations.

### Imputation - important

Imputation should be done over the training set, and then propagated to the test set. This means that the mean / median to be used to fill missing values both in train and test set, should be extracted from the train set only. And this is to avoid overfitting.

First, let's separate the data into training and testing set.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    data[['Age', 'Fare']],  # predictors
    data['Survived'],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

Let's explore the missing data in the train set. The percentages should be fairly similar to those of the whole dataset.

In [0]:
X_train.isnull().mean()

Let's make a function to fill missing values with the mean or median:

In [0]:
# The function takes the dataframe, the variable, and the value of the mean or median as parameters and returns the variable with the filled na.


def impute_na(df, variable, mean_median):

    return df[variable].fillna(mean_median)

**Exercise:** What is the mean and median age?

In [0]:
# calculate the median Age in X train


# calculate the mean Age in X trai


# print the information


Let's create a new variable with the missing values replaced, using the function we created above

In [0]:
# first replace with the median
X_train['Age_median'] = impute_na(X_train, 'Age', median)

# now replace with the mean
X_train['Age_mean'] = impute_na(X_train, 'Age', mean)

# the mean contains many decimals, so I round to 1, using # the round function from numpy
X_train['Age_mean'] = np.round(X_train['Age_mean'], 0)
X_train.head(15)

Look at the rows with missing data (NaN) in Age, and see how in the new variables those were replaced by either 29 (median) or 30 (mean).

Let's see if there is a variance change after the imputation.

In [0]:
print('Original variable variance: ', X_train['Age'].var())
print('Variance after median imputation: ', X_train['Age_median'].var())
print('Variance after mean imputation: ', X_train['Age_mean'].var())

We can see a change in the variance after mean / median imputation. This is expected, because the percentage of missing data is quite high in Age, ~20%.

As expected, the variance is underestimated, because now many values are the same ==> either the mean or the median value. We can see through graphs that the distribution has changed, accumulating more values towards the median or median.

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111)

# original variable distribution
X_train['Age'].plot(kind='kde', ax=ax)

# variable imputed with the median
X_train['Age_median'].plot(kind='kde', ax=ax, color='red')

# variable imputed with the mean
X_train['Age_mean'].plot(kind='kde', ax=ax, color='green')

# add legends
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')

As mentioned above, the mean / median imputation distorts the original distribution of the variable Age. The transformed variable shows more values around the mean / median values.

**Is this important?**

It depends on the machine learning model you want to build. Linear models assume that the variables are normally distributed. Mean / median imputation may distort the original normal distribution if the % of missing data is high. Therefore the final imputed variable will no longer be normally distributed, which in turn may affect the linear model performance.

Finally, we mentioned that mean / median imputation may lead to observations that are normal, to look like outliers or in other words, mean / median imputation may lead to an increase in the apparent number of  outliers. Let's use a boxplot to see if it happens.

In [0]:
X_train[['Age', 'Age_median', 'Age_mean']].boxplot()

From the boxplot above, we can see that after the imputation not only we have more outliers on the higher Age values, but we have now outliers as well for the lower values of Age.

**Is this important?**

If we are after true outliers, we need to keep this behaviour in mind, to make sure that we are neither masking nor creating artificial outliers with our imputation technique. In practice, we normally don't check for this behaviour at all. But I think it is important to know that is happening.

**Authors:** Juliana da Mota Coelho, Camila Mizokami. 

**Adapted by** Kamilla Silva

**References:**

https://pandas.pydata.org/docs/user_guide/missing_data.html 

https://chartio.com/resources/tutorials/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe/ 

https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html 

https://towardsdatascience.com/3-ultimate-ways-to-deal-with-missing-values-in-python-ac5a17c53787